In [ ]:
# https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/scikit_learn_inference_pipeline

In [11]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.sklearn import SKLearnModel
from sagemaker.xgboost import XGBoostModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker import PipelineModel

In [12]:
sagemaker_session = sagemaker.Session()

sklearn_model = SKLearnModel(
    # next line is something like a hack. We configure model_data, even though we need to execute only pre-processing.
    # Reason for this is the validation executed by SageMaker. As such we define the xgboost model path or a dummy path in general
    # to pass throught the validation step. Model obviously is never applied.
    model_data='s3://ktzouvan-trading-point-sagemaker-poc/models/pipelines-3ol04e0v16ml-training-step-sALSc8gqXi/output/model.tar.gz',
    entry_point='0_sklearn_inference.py',
    framework_version='0.23-1',  # Ensure this matches your SageMaker SKLearn container version
    role=sagemaker.get_execution_role(),
    sagemaker_session=sagemaker_session
)

xgboost_model = XGBoostModel(
   model_data='s3://ktzouvan-trading-point-sagemaker-poc/models/pipelines-3ol04e0v16ml-training-step-sALSc8gqXi/output/model.tar.gz',
   entry_point='0_xgboost_inference.py',
   framework_version='1.7-1',
   role=sagemaker.get_execution_role(),
   sagemaker_session=sagemaker_session
)

pipeline_model = PipelineModel(
   name='preprocessXGBboost',
   models=[sklearn_model, xgboost_model],
   role=sagemaker.get_execution_role(),
   sagemaker_session=sagemaker_session,
)

In [ ]:
pipeline_model.deploy(
    endpoint_name= 'preprocessXGBboost',
    initial_instance_count=1,
    instance_type='ml.m5.xlarge'
)

------